In [1]:
import numpy as np
from numba import njit, types

import re
from functools import reduce
from tqdm import tqdm
from pathlib import Path
pathtohere = Path.cwd()
print(pathtohere)

C:\Users\02dba\Documents\GitHub\Diatomic-Molecule-Dissociation-by-Gravitational-Gradient\contents


In [2]:
def simplifyExpressions(motionEquations:'list'):
    """
    Apply the compression algorithm to each of the four dof.
    
    Inputs:
        - motionEquations:list: Expressions for each of the dofs.
        
    Outputs:
        - variables:list: Expressions that have been substituted.
        - motionEquations:list: New expressions with the substitutions.
    """
    
    variables = [[] for i in range(len(motionEquations))]
    
    for i in range(len(motionEquations)):
        print(f'{i+1}/{len(motionEquations)}')
        variables[i], motionEquations[i] = simplifyExpression(motionEquations[i])
        
    return variables, motionEquations


@njit('b1(types.unicode_type)')
def testBrackets(testString:str) -> bool:
    """
    Test to see if all brackets are closed in a string.
    
    Inputs:
        - testString:str: String to test.
        
    Outputs:
        - isClosed:bool: Whether the brackets are closed.
    """
    # Perform checks to see if it is a valid subexpression.
    bracketLevel = 0
    
    for char in testString[1:-1]:
        if char=='(':
            bracketLevel += 1
        elif char==')':
            bracketLevel -= 1
            if bracketLevel == -1:
                return False
    
    return bracketLevel == 0
    
    

def simplifyExpression(expression:str, lowerLength:int=15, upperLength:int=1_000,
                      runNumber:int=0, numTemporary:int=0, tempNameLen:int=9):
    """
    Extract common parts of a C-style expression taking parentheses into account.
    
    Inputs:
        - expression:str: Expression to simplify.
        - lowerLength:int: Lower bound of the length of substring to test.
        - upperLength:int: Upper bound of the length of substring to test.
        - runNumber:int: Number of times the compression procedure has been run.
        - numTemporary:int: Number of temporary variables.
        - tempNameLen:int: Length of the temporary variable.
        
    Outputs:
        - variables:list: Expressions that have been substituted.
        - expression:str: New expression with the substitutions.
    """
    
    
    operators_plus = {'+','-'}
    operators_mul = {'*',}

    stringReplaceThreshold = 200
    numPasses = 10
    if upperLength>len(expression)//2: upperLength = len(expression)//2

    variables = []
    i = 0
    
    while i < len(expression) - lowerLength:
        if (i+1)%((len(expression) - lowerLength)//10)==0 and runNumber==0:
            print(f'Progress: {100*(i+1)/(len(expression) - lowerLength):0.1f}%')
        
#         for j in range(0,upperLength - lowerLength):
#             testString = expression[i:i+upperLength-j]
        for j in range(lowerLength, upperLength):
            testString = expression[i:i+j]
                        
            # +...+ / -...-
            if (not (testString[0] in operators_plus and testString[-1] in operators_plus 
                and (any(x in testString[1:-1] for x in operators_plus) 
                     or any(x in testString[1:-1] for x in operators_mul))
                and testBrackets(testString)
                #and testString[1]!='>' and expression[i+upperLength-j]!='>')
                and testString[1]!='>' and expression[i+j]!='>')
            # *...*
            and not ((testString[0] in operators_mul or testString[0] in operators_plus) 
                and (testString[-1] in operators_mul or testString[-1] in operators_plus)
                and any(x in testString[1:-1] for x in operators_mul)
                and '+' not in testString[1:-1]
                and testString[1:-1].count('-') <= testString[1:-1].count('>')
                and testBrackets(testString)
                #and testString[1]!='>' and expression[i+upperLength-j]!='>')
                and testString[1]!='>' and expression[i+j]!='>')
            # (...)
           and not (testString[0]=='(' and testString[-1]==')'
                and (any(x in testString[1:-1] for x in operators_plus) 
                     or any(x in testString[1:-1] for x in operators_mul))
                and ',' not in testString
                and testBrackets(testString))):
                continue


                

            
            # Find instances of substring in expression.
            inds = [m.start() for m in re.finditer(re.escape(testString), expression)]
            if len(inds)>1 or (len(testString)<stringReplaceThreshold 
                               and runNumber!=0):
                
                numTemporary += 1

                if numTemporary==11 or numTemporary==101 or numTemporary==1001:
                    tempNameLen += 1
                    assert lowerLength > tempNameLen

                lenTest = len(testString)
                
                
                # Perform substitutions.
                for numSub,k in enumerate(inds):
                    adj = numSub*(tempNameLen - lenTest)
                    
                    if testString[0]=='(' and testString[-1]==')':
                        adj -= 2*numSub
                        expression = (expression[:k + adj] 
                                    + f'temp[{numTemporary-1}]' 
                                      + expression[k + adj + lenTest:])
                    else:
                        expression = (expression[:k + adj] 
                                    + f'{testString[0]}temp[{numTemporary-1}]{testString[-1]}' 
                                      + expression[k + adj + lenTest:])
                
                    
                i += lenTest
                
                
                # Invert temporary if is preceeded by a minus.
                if testString[0]=='-':
                    for k in range(1,lenTest-1):
                        if testString[k]=='+':
                            testString = testString[:k] + '-' + testString[k+1:]
                        elif testString[k]=='-' and testString[k+1]!='>':
                            testString = testString[:k] + '+' + testString[k+1:]
                
                variables.append(testString[1:-1])

                
                break
                    
        i += 1
                
    if runNumber < numPasses:
        varNew, expression = simplifyExpression(expression, runNumber = runNumber + 1,
                                            numTemporary=numTemporary, tempNameLen=tempNameLen)
    
        variables += varNew

        
    assert len(variables) < 10001, f'Too many temporary variables: {len(variables)}>10001'
        
    return variables, expression
                
                

In [3]:
def formatExpressions(motionEquations:'list'):
    """
    Replace parts of the expression to make it more efficient and compatable with the already present variables.
    
    Inputs:
        - motionEquations:list: Expressions for each of the dofs.
        
    Outputs:
        - motionEquations:list: New expressions for each of the dofs.
    
    """
    
    patterns = {
        'q' : 'p->charge',
        'sigma(r,theta)': 'sigma_p',
        'delta(r)': 'delta_p',
        'chi(theta)': 'chi_p',
        'Power(Q,2)': 'BH->charge2',
        'Power(l,2)': 'BH->l2',
        'Power(a,2) + Power(l,2) + Power(r,2)': 'precalculated->ral_sqr',
        'Q': 'BH->charge',
        'l': 'BH->l',
        'a': 'BH->a',
        'M': 'BH->mass',
        'Power(a,2)': 'BH->a2',
        'Power(l,2)': 'BH->l2',
        'Power(Q,2)': 'BH->charge2',
        'Power(l,3)': 'BH->l3',
        'Power(a,3)': 'BH->a3',
        'Power(a,4)': 'BH->a4',
        'Power(l,4)': 'BH->l4',
        'Power(a,5)': 'BH->a5',
        'Power(l,5)': 'BH->l5',
        'Power(a,6)': 'BH->a6',
        'Power(l,6)': 'BH->l6',
        'Power(a,7)': 'BH->a7',
        'Power(l,7)': 'BH->l7',
        'Power(a,8)': 'BH->a8',
        'Power(l,8)': 'BH->l8',
        'Power(a,9)': 'BH->a9',
        'Power(l,9)': 'BH->l9',
        'Power(a,10)': 'BH->a10',
        'Cos': 'cos',
        'Sin': 'sin',
        'Csc': '1. / sin',
        'Cot': '1. / tan',
        'Sec': '1. / cos',
        'm' : 'p->mass',
        'r': 'p->r',
        'phi': 'p->phi',
        'theta': 'p->theta',
        'Power(q,2)': '(p->charge*ps->charge)',
        'angular' : 'p->L',
        'energy' : 'p->energy',
        'Power(r,2)': 'p->r2',
        'Power(r,3)': 'p->r3',
        'Power(r,4)': 'p->r4',
        'Power(r,5)': 'p->r5',
        'Power(r,6)': 'p->r6',
        'Power(r,7)': 'p->r7',
        'rs': 'ps->r',
        'phis': 'ps->phi',
        'thetas': 'ps->theta',
        'Power(rdot,2)': '(precalculated->rdot*precalculated->rdot)',
        'Power(phidot,2)': '(precalculated->phidot*precalculated->phidot)',
        'Power(thetadot,2)': '(precalculated->thetadot*precalculated->thetadot)',
        
        'Power(sigma(r,theta),2)' : 'precalculated->sigma_p2',
        'Power(sigma(r,theta),3)' : 'precalculated->sigma_p3',
        'Power(sigma(r,theta),4)' : 'precalculated->sigma_p4',
        'Power(delta(r),2)' : 'precalculated->delta_p2',
        'Power(delta(r),3)' : 'precalculated->delta_p3',
        'Power(delta(r),4)' : 'precalculated->delta_p4',
        'Power(chi(theta),2)' : 'precalculated->chi_p2',
        'Power(chi(theta),3)' : 'precalculated->chi_p3',
        'Power(chi(theta),4)' : 'precalculated->chi_p4',
        
        'Cos(theta)': 'precalculated->costheta',
        'Sin(theta)': 'precalculated->sintheta',
        'Sin(2.*theta)': 'precalculated->sin2theta',
        'Power(Sin(theta),2)': 'precalculated->sintheta2',
        'Power(Sin(theta),3)': 'precalculated->sintheta3',
        'Power(Sin(theta),4)': 'precalculated->sintheta4',
        'Power(Sin(theta),5)': 'precalculated->sintheta5',
        'Power(Sin(theta),6)': 'precalculated->sintheta6',
        'Power(Sin(theta),7)': 'precalculated->sintheta7',
        'Sin(phi)': 'precalculated->sinphi',
        'Cos(phi)': 'precalculated->cosphi',
        
        # Source stuff.
        'Sin(thetas)': 'precalculated->sinthetas',
        'Cos(thetas)': 'precalculated->costhetas',
        'Sin(phis)': 'precalculated->sinphis',
        'Cos(phis)': 'precalculated->cosphis',
        'Sin(phisdot)': 'precalculated->sinphisdot',
        'Cos(phisdot)': 'precalculated->cosphisdot',
        'Sin(thetasdot)': 'precalculated->sinthetasdot',
        'Cos(thetasdot)': 'precalculated->costhetasdot',
        'Power(Sin(thetasdot),2)': 'precalculated->sinthetas2',
        'Power(Cos(thetasdot),2)': 'precalculated->costhetas2',
        'Power(Sin(phisdot),2)': 'precalculated->sinphis2',
        'Power(Cos(phisdot),2)': 'precalculated->cosphis2',
        'Power(rsdot,2)': 'precalculated->rsdot2',
        'Power(phisdot,2)': 'precalculated->phisdot2',
        
        'epsilon': 'ps->epsilon',
        'sigma0': 'ps->sigma',
        'Power(sigma0,6)': 'ps->sigma6',
        'Power(sigma0,12)': 'ps->sigma12',
        
        # n vector
        'Power(nmag,2)': 'precalculated->nmag2',
        'Power(nmag,5)': 'precalculated->nmag5',
        'Power(nmag,6)': 'precalculated->nmag6',
        'Power(nmag,7)': 'precalculated->nmag7',
        'Power(nmag,8)': 'precalculated->nmag8',
        'Power(nmag,12)': 'precalculated->nmag12',
        'Power(nmag,13)': 'precalculated->nmag13',
        'Power(nmag,14)': 'precalculated->nmag14',
        'Power(nx,2)': '(precalculated->nx*precalculated->nx)',
        'Power(ny,2)': '(precalculated->ny*precalculated->ny)',
        'Power(nz,2)': '(precalculated->nz*precalculated->nz)',
        # Keep the same.
        '1.*' : '',
        '2.' : '2.',
        '/2' : '/2.',
        'Power' : 'pow',
        'epsilon0' : 'epsilon_0',
        'nmag' : 'precalculated->nmag',
        'tdot': 'precalculated->tdot',
        'rdot' : 'precalculated->rdot',
        'phidot' : 'precalculated->phidot',
        'thetadot' : 'precalculated->thetadot',
        'tsdot': 'precalculated->tsdot',
        'rsdot' : 'precalculated->rsdot',
        'phisdot' : 'precalculated->phisdot',
        'thetasdot' : 'precalculated->thetasdot',
        'tsddot':'precalculated->tsddot',
        'rsddot' : 'precalculated->rsddot',
        'phisddot':'precalculated->phisddot',
        'thetasddot' : 'precalculated->thetasddot',
        'nx' : 'precalculated->nx',
        'ny' : 'precalculated->ny',
        'nz' : 'precalculated->nz',
        
        # Plane case.
        'sigma(r,Pi/2.)': 'sigma_p',
        'chi(Pi/2.)': 'chi_p',
        'Power(sigma(r,Pi/2.),2)' : 'precalculated->sigma_p2',
        'Power(sigma(r,Pi/2.),3)' : 'precalculated->sigma_p3',
        'Power(sigma(r,Pi/2.),4)' : 'precalculated->sigma_p4',
        'Power(chi(Pi/2.),2)' : 'precalculated->chi_p2',
        'Power(chi(Pi/2.),3)' : 'precalculated->chi_p3',
        'Power(chi(Pi/2.),4)' : 'precalculated->chi_p4',
    }
    
    maxPatternLength = max(map(len, patterns.keys()))
    
    for i in range(len(motionEquations)):
        j = 0
        while j < len(motionEquations[i]):
            # Run from largest pattern lengt to smallest.
            for k in range(0,maxPatternLength):
                section = motionEquations[i][j:j+maxPatternLength - k] 
                
                # Replace part.
                if section in patterns.keys():
                    motionEquations[i] = (motionEquations[i][:j] + patterns[section] 
                                        + motionEquations[i][j+maxPatternLength - k:])
                    j += len(patterns[section]) - 1
                    break
                    
            j += 1
            
        # Remove spaces.
        motionEquations[i] = motionEquations[i].replace(" ", "")
                
                
                    
    return motionEquations


def writeIntermediateVariables_h(f):
    """
    Write intermeditate variables to the header (.h) file f.
    
    Inputs:
        - f: Instance of the file.
    """

    f.write('struct Precalculated{\n')
    f.write('\tconst cpp_dec_float_25 sigma_p2{};\n')
    f.write('\tconst cpp_dec_float_25 sigma_p3{};\n')
    f.write('\tconst cpp_dec_float_25 sigma_p4{};\n')
    f.write('\tconst cpp_dec_float_25 delta_p2{};\n')
    f.write('\tconst cpp_dec_float_25 delta_p3{};\n')
    f.write('\tconst cpp_dec_float_25 delta_p4{};\n')
    f.write('\tconst cpp_dec_float_25 chi_p2{};\n')
    f.write('\tconst cpp_dec_float_25 chi_p3{};\n')
    f.write('\tconst cpp_dec_float_25 chi_p4{};\n\n')

    f.write('\tconst cpp_dec_float_25 tdot{};\n')
    f.write('\tconst cpp_dec_float_25 rdot{};\n')
    f.write('\tconst cpp_dec_float_25 phidot{};\n')
    f.write('\tconst cpp_dec_float_25 thetadot{};\n\n')
    f.write('\tconst cpp_dec_float_25 tsdot{};\n')
    f.write('\tconst cpp_dec_float_25 rsdot{};\n')
    f.write('\tconst cpp_dec_float_25 phisdot{};\n')
    f.write('\tconst cpp_dec_float_25 thetasdot{};\n')
    f.write('\tconst cpp_dec_float_25 tsddot{};\n')
    f.write('\tconst cpp_dec_float_25 rsddot{};\n')
    f.write('\tconst cpp_dec_float_25 phisddot{};\n')
    f.write('\tconst cpp_dec_float_25 thetasddot{};\n\n')

    f.write('\tconst cpp_dec_float_25 nx{};\n') 
    f.write('\tconst cpp_dec_float_25 ny{};\n') 
    f.write('\tconst cpp_dec_float_25 nz{};\n')
    f.write('\tconst cpp_dec_float_25 nmag{};\n')
    f.write('\tconst cpp_dec_float_25 nmag2{};\n')
    f.write('\tconst cpp_dec_float_25 nmag5{};\n')
    f.write('\tconst cpp_dec_float_25 nmag6{};\n')
    f.write('\tconst cpp_dec_float_25 nmag7{};\n')
    f.write('\tconst cpp_dec_float_25 nmag8{};\n')
    f.write('\tconst cpp_dec_float_25 nmag12{};\n')
    f.write('\tconst cpp_dec_float_25 nmag13{};\n')
    f.write('\tconst cpp_dec_float_25 nmag14{};\n\n')


    f.write('\tconst cpp_dec_float_25 ral_sqr{};\n')
    f.write('\tconst cpp_dec_float_25 sintheta{};\n')
    f.write('\tconst cpp_dec_float_25 sintheta2{};\n')
    f.write('\tconst cpp_dec_float_25 sintheta3{};\n')
    f.write('\tconst cpp_dec_float_25 sintheta4{};\n')
    f.write('\tconst cpp_dec_float_25 sintheta5{};\n')
    f.write('\tconst cpp_dec_float_25 sintheta6{};\n')
    f.write('\tconst cpp_dec_float_25 sintheta7{};\n\n')

    f.write('\tconst cpp_dec_float_25 costheta{};\n')
    f.write('\tconst cpp_dec_float_25 sin2theta{};\n')
    f.write('\tconst cpp_dec_float_25 sinphi{};\n')
    f.write('\tconst cpp_dec_float_25 cosphi{};\n\n')

    f.write('\tconst cpp_dec_float_25 sinthetas{};\n')
    f.write('\tconst cpp_dec_float_25 costhetas{};\n')
    f.write('\tconst cpp_dec_float_25 sinphis{};\n')
    f.write('\tconst cpp_dec_float_25 cosphis{};\n')
    f.write('\tconst cpp_dec_float_25 sinthetasdot{};\n')
    f.write('\tconst cpp_dec_float_25 costhetasdot{};\n')
    f.write('\tconst cpp_dec_float_25 sinphisdot{};\n')
    f.write('\tconst cpp_dec_float_25 cosphisdot{};\n')
    f.write('\tconst cpp_dec_float_25 sinthetas2{};\n')
    f.write('\tconst cpp_dec_float_25 costhetas2{};\n')
    f.write('\tconst cpp_dec_float_25 sinphis2{};\n')
    f.write('\tconst cpp_dec_float_25 cosphis2{};\n')
    f.write('\tconst cpp_dec_float_25 rsdot2{};\n')
    f.write('\tconst cpp_dec_float_25 phisdot2{};\n\n')
    
    f.write('Precalculated(const BlackHole* BH,\n')
    f.write('\t\tParticle* p, Particle* ps,\n')
    f.write('\t\tconst cpp_dec_float_25 sigma_p, const cpp_dec_float_25 delta_p,\n')
    f.write('\t\tconst cpp_dec_float_25 chi_p, const cpp_dec_float_25 dlambda);\n')
    
    f.write('};\n\n')
    
def writeIntermediateVariables_cpp(f):
    """
    Write intermeditate variables to the .cpp file f.
    
    Inputs:
        - f: Instance of the file.
    """
    
    f.write('Precalculated::Precalculated(const BlackHole* BH,\n')
    f.write('\tParticle* p, Particle* ps,\n')
    f.write('\tconst cpp_dec_float_25 sigma_p, const cpp_dec_float_25 delta_p,\n')
    f.write('\tconst cpp_dec_float_25 chi_p, const cpp_dec_float_25 dlambda):\n')
        
    f.write('\tsigma_p2{ sigma_p*sigma_p },\n')
    f.write('\tsigma_p3{ sigma_p*sigma_p2 },\n')
    f.write('\tsigma_p4{ sigma_p2*sigma_p2 },\n')
    f.write('\tdelta_p2{ delta_p*delta_p },\n')
    f.write('\tdelta_p3{ delta_p*delta_p2 },\n')
    f.write('\tdelta_p4{ delta_p2*delta_p2 },\n')
    f.write('\tchi_p2{ chi_p*chi_p },\n')
    f.write('\tchi_p3{ chi_p*chi_p2 },\n')
    f.write('\tchi_p4{ chi_p2*chi_p2 },\n\n')

    f.write('\ttdot{ (p->t - p->t_prev) / dlambda },\n')
    f.write('\trdot{ (p->r - p->r_prev) / dlambda },\n')
    f.write('\tphidot{ (p->phi - p->phi_prev) / dlambda },\n')
    f.write('\tthetadot{ (p->theta - p->theta_prev) / dlambda },\n\n')
    f.write('\ttsdot{ (ps->t - ps->t_prev) / dlambda },\n')
    f.write('\trsdot{ (ps->r - ps->r_prev) / dlambda },\n')
    f.write('\tphisdot{ (ps->phi - ps->phi_prev) / dlambda },\n')
    f.write('\tthetasdot{ (ps->theta - ps->theta_prev) / dlambda },\n')
    f.write('\ttsddot{\n')
    f.write('\t\t (ps->t - 2.*ps->t_prev + ps->t_prevprev) / (dlambda*dlambda)},\n')
    f.write('\trsddot{\n')
    f.write('\t\t(ps->r - 2.*ps->r_prev + ps->r_prevprev) / (dlambda*dlambda)},\n')
    f.write('\tphisddot{\n')
    f.write('\t\t(ps->phi - 2.*ps->phi_prev + ps->phi_prevprev) / (dlambda*dlambda)},\n')
    f.write('\tthetasddot{\n')
    f.write('\t\t(ps->theta - 2.*ps->theta_prev + ps->theta_prevprev) / (dlambda*dlambda)},\n\n')

    f.write('\tnx{p->r*sin(p->theta)*cos(p->phi) - ps->r*sin(ps->theta)*cos(ps->phi)},\n') 
    f.write('\tny{p->r*sin(p->theta)*sin(p->phi) - ps->r*sin(ps->theta)*sin(ps->phi)},\n') 
    f.write('\tnz{p->r*cos(p->theta) - ps->r*cos(ps->theta)},\n')
    f.write('\tnmag{sqrt(nx*nx + ny*ny + nz*nz)},\n')
    f.write('\tnmag2{nmag*nmag},\n')
    f.write('\tnmag5{nmag*nmag2*nmag2},\n')
    f.write('\tnmag6{nmag5*nmag},\n')
    f.write('\tnmag7{nmag6*nmag},\n')
    f.write('\tnmag8{nmag6*nmag2},\n')
    f.write('\tnmag12{nmag6*nmag6},\n')
    f.write('\tnmag13{nmag6*nmag7},\n')
    f.write('\tnmag14{nmag7*nmag7},\n\n')


    f.write('\tral_sqr{BH->a2 + BH->l2 + p->r2},\n')
    f.write('\tsintheta{sin(p->theta)},\n')
    f.write('\tsintheta2{sintheta*sintheta},\n')
    f.write('\tsintheta3{sintheta2*sintheta},\n')
    f.write('\tsintheta4{sintheta2*sintheta2},\n')
    f.write('\tsintheta5{sintheta2*sintheta3},\n')
    f.write('\tsintheta6{sintheta3*sintheta3},\n')

    f.write('\t\tsintheta7{sintheta3*sintheta4},\n\n')
    f.write('\tcostheta{cos(p->theta)},\n')
    f.write('\tsin2theta{sin(2.*p->theta)},\n')
    f.write('\tsinphi{sin(p->phi)},\n')
    f.write('\tcosphi{cos(p->phi)},\n\n')

    f.write('\tsinthetas{sin(ps->theta)},\n')
    f.write('\tcosthetas{cos(ps->theta)},\n')
    f.write('\tsinphis{sin(ps->phi)},\n')
    f.write('\tcosphis{cos(ps->phi)},\n')
    f.write('\tsinthetasdot{sin(thetasdot)},\n')
    f.write('\tcosthetasdot{cos(thetasdot)},\n')
    f.write('\tsinphisdot{sin(phisdot)},\n')
    f.write('\tcosphisdot{cos(phisdot)},\n')
    f.write('\tsinthetas2{sinthetas*sinthetas},\n')
    f.write('\tcosthetas2{costhetas*costhetas},\n')
    f.write('\tsinphis2{sinphis*sinphis},\n')
    f.write('\tcosphis2{cosphis*cosphis},\n')
    f.write('\trsdot2{rsdot*rsdot},\n')
    f.write('\tphisdot2{phisdot*phisdot}\n')
    f.write('{\n')
    
    f.write('\t/*\n')
    f.write('\tInitalise the struct.\n')
    f.write('\t*/\n')
    f.write('\treturn;\n')
    f.write('}\n')


def configureFile():
    """
    Configure the cpp file to find the new coordinates from the Euler-Lagrange equations.
    """
    
    configureIdentifier = '// Configured==true\n'
    
    motionEquations = []
    
    line = True
    filePath = pathtohere / 'src/eulerLagrange.cpp'
    i = 0

    # Read in the expressions.
    # https://stackoverflow.com/questions/3277503/how-to-read-a-file-line-by-line-into-a-list
    with open(filePath, 'r', encoding='UTF-8') as f:
        while line and i < 4:
            line = f.readline()
            
            if line == configureIdentifier:
                print(f'File already configured.')
                return
            if i==0:
                print(f'File not configured: {configureIdentifier[:-1]} not found.')
            
            
            cut = slice(0,-1) if i<3 else slice(0,None)
            motionEquations.append(line[cut])
            
            i+=1
            
    # Manipulate expressions.
    motionEquations = formatExpressions(motionEquations)
    
    variables, motionEquations = simplifyExpressions(motionEquations)
    
    dofs = ('t','r','phi','theta')
    
    # Write to .cpp file.
    with open(filePath, 'w', encoding='UTF-8') as f:
        f.write(f'{configureIdentifier}')
        f.write("#include \"eulerLagrange.h\"\n\n")

        f.write('const cpp_dec_float_25 epsilon_0{ 1. / (4.*M_PI) }; // Geometrised-Gaussian units\n\n\n')
        
        writeIntermediateVariables_cpp(f)
        
        for i in range(len(motionEquations)):
            f.write(f'cpp_dec_float_25 calc_{dofs[i]}_next(const BlackHole* BH,\n')
            f.write('\tParticle* p, Particle* ps,\n')
            f.write('\tconst cpp_dec_float_25 sigma_p, const cpp_dec_float_25 delta_p,\n')
            f.write('\tconst cpp_dec_float_25 chi_p, const Precalculated* precalculated,\n')
            f.write('\tconst cpp_dec_float_25 dlambda)\n')
            f.write('{\n')
            f.write('\t/*\n')
            f.write(f'\tCalculate the next value of {dofs[i]}.\n\n')
            f.write('\tInputs:\n')
            f.write('\t\t- const BlackHole* BH: Instance of the black hole.\n')
            f.write('\t\t- Particle* p: Instance of the particle for which the next coord is being calculated.\n')
            f.write('\t\t- Particle* ps: Instance of the source particle.\n')
            f.write('\t\t- const cpp_dec_float_25 sigma_p: Precalculated sigma value.\n')
            f.write('\t\t- const cpp_dec_float_25 delta_p: Precalculated delta value.\n')
            f.write('\t\t- const cpp_dec_float_25 chi_p: Precalculated chi value.\n')
            f.write('\t\t- const Precalculated* precalculated: Instance of the precalculated variables struct.\n')
            f.write('\t\t- const cpp_dec_float_25 dlambda: Lambda step.\n\n')
            
            f.write('\tOutputs:\n')
            f.write(f'\t\t- cpp_dec_float_25 {dofs[i]}_new: New coordinate.\n')
            f.write('\t*/\n\n')
            
            
            f.write(f'\tcpp_dec_float_25 {dofs[i]}_new' + '{};\n')

            if len(variables[i]) != 0:
                f.write(f'\tcpp_dec_float_25 temp[{len(variables[i])}];\n')
                for j in range(len(variables[i])):
                    f.write('\ttemp[' + str(j) + '] = ' + variables[i][j] + ';\n')
            f.write('\n\n')

            f.write(f'\t{dofs[i]}_new = 2. * p->{dofs[i]} - p->{dofs[i]}_prev + dlambda*dlambda*({motionEquations[i]});\n')
            f.write(f'\treturn {dofs[i]}_new;\n')
            f.write('}\n\n')

        
        f.write('std::tuple<cpp_dec_float_25, cpp_dec_float_25,\n')
        f.write('\tcpp_dec_float_25, cpp_dec_float_25> eulerMoveMathematica(const BlackHole* BH,\n')
        f.write('\tParticle* p, Particle* ps,\n')
        f.write('\tconst cpp_dec_float_25 sigma_p, const cpp_dec_float_25 delta_p,\n')
        f.write('\tconst cpp_dec_float_25 chi_p, const cpp_dec_float_25 dlambda)\n')
        f.write('{\n')
        f.write('\t/*\n')
        f.write(f'\tCalculate the next value of {dofs[i]}.\n\n')
        f.write('\tInputs:\n')
        f.write('\t\t- const BlackHole* BH: Instance of the black hole.\n')
        f.write('\t\t- Particle* p: Instance of the particle for which the next coord is being calculated.\n')
        f.write('\t\t- Particle* ps: Instance of the source particle.\n')
        f.write('\t\t- const cpp_dec_float_25 sigma_p: Precalculated sigma value.\n')
        f.write('\t\t- const cpp_dec_float_25 delta_p: Precalculated delta value.\n')
        f.write('\t\t- const cpp_dec_float_25 chi_p: Precalculated chi value.\n')
        f.write('\t\t- const cpp_dec_float_25 dlambda: Lambda step.\n\n')

        f.write('\tOutputs:\n')
        f.write(f'\t\t- cpp_dec_float_25 t_new: New t coordinate.\n')
        f.write(f'\t\t- cpp_dec_float_25 r_new: New r coordinate.\n')
        f.write(f'\t\t- cpp_dec_float_25 phi_new: New phi coordinate.\n')
        f.write(f'\t\t- cpp_dec_float_25 theta_new: New theta coordinate.\n')
        f.write('\t*/\n\n')
        f.write('\tcpp_dec_float_25 t_new{};\n')
        f.write('\tcpp_dec_float_25 r_new{};\n')
        f.write('\tcpp_dec_float_25 phi_new{};\n')
        f.write('\tcpp_dec_float_25 theta_new{};\n\n')
        
        f.write('\tPrecalculated precalculated{BH, p, ps, sigma_p, delta_p, chi_p, dlambda};\n\n')
        
        
        for i in range(len(motionEquations)):
            f.write(f'\t{dofs[i]}_new = calc_{dofs[i]}_next(BH,\n')
            f.write('\t\tp, ps,\n')
            f.write('\t\tsigma_p, delta_p,\n')
            f.write('\t\tchi_p, &precalculated, dlambda);\n\n')
        
        
#         for i in range(len(motionEquations)):
#             for j in range(len(variables[i])):
#                 f.write('\tconst cpp_dec_float_25 temp' + str(i) + str(j) + '{ ' + variables[i][j] + ' };\n')
#         f.write('\n\n')
                
        
#         f.write(f'\tt_new = 2. * p->t - p->t_prev + dlambda*dlambda*({motionEquations[0]});\n')
#         f.write(f'\tr_new = 2. * p->r - p->r_prev + dlambda*dlambda*({motionEquations[1]});\n')
#         f.write(f'\tphi_new = 2. * p->phi - p->phi_prev + dlambda*dlambda*({motionEquations[2]});\n')
#         f.write(f'\ttheta_new = 2. * p->theta - p->theta_prev + dlambda*dlambda*({motionEquations[3]});\n\n')

        
        f.write('\treturn {t_new, r_new, phi_new, theta_new};\n')
        f.write('}\n')
        
        
    # Write to .h file.
    filePath = pathtohere / 'include/eulerLagrange.h'
    with open(filePath, 'w', encoding='UTF-8') as f:
        f.write('#ifndef EULERLAGRANGE_H\n')
        f.write('#define EULERLAGRANGE_H\n\n')
        
        f.write('#include <iostream>\n')
        f.write('#include <tuple>\n')
        f.write('#define _USE_MATH_DEFINES\n')
        f.write('#include "math.h"\n\n')
        f.write('#include <boost/multiprecision/cpp_dec_float.hpp>\n')
        f.write('using namespace boost::multiprecision;\n')
        f.write('using cpp_dec_float_25 = number<cpp_dec_float<25>>;\n\n')


        f.write('#include "blackHole.h"\n')
        f.write('#include "particle.h"\n\n\n')
        
        
        for i in range(len(motionEquations)):
            f.write(f'cpp_dec_float_25 calc_{dofs[i]}_next(const BlackHole* BH,\n')
            f.write('\tParticle* p, Particle* ps,\n')
            f.write('\tconst cpp_dec_float_25 sigma_p, const cpp_dec_float_25 delta_p,\n')
            f.write('\tconst cpp_dec_float_25 chi_p, const cpp_dec_float_25 dlambda);\n\n')
        
        writeIntermediateVariables_h(f)
        
        f.write('std::tuple<cpp_dec_float_25, cpp_dec_float_25,')
        f.write('\tcpp_dec_float_25, cpp_dec_float_25> eulerMoveMathematica(const BlackHole* BH,\n')
        f.write('\tParticle* p, Particle* ps,\n')
        f.write('\tconst cpp_dec_float_25 sigma_p, const cpp_dec_float_25 delta_p,')
        f.write('\tconst cpp_dec_float_25 chi_p, const cpp_dec_float_25 dlambda);\n\n')

        f.write('#endif EULERLAGRANGE_H\n')
        
    print('Configuration complete.')
            
                        
            

In [4]:
def main():
    configureFile()

In [5]:
if __name__=='__main__':
    main()

File not configured: // Configured==true not found.
1/4
Progress: 10.0%
Progress: 20.0%
Progress: 69.9%
Progress: 79.8%
Progress: 99.9%
2/4
Progress: 10.0%
Progress: 100.0%
3/4
Progress: 10.0%
Progress: 50.0%
Progress: 60.0%
Progress: 70.0%
Progress: 80.0%
Progress: 99.9%
4/4
Progress: 10.0%
Progress: 59.7%
Progress: 69.6%
Progress: 99.2%
Configuration complete.
